# Import Library

In [ ]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
import random
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer


# Read the JSON data

In [ ]:
file_path ='new_data_Ai.json'
file_path = r"C:\Users\asus\Desktop\Artificial intelligence\Third Year\Scaned S\Ai-417 Deep\project\new_data_Ai.json"
with open(file_path, 'r') as file:
    intents = json.load(file)


# Tokenize & Stem 

In [ ]:
stemmer = PorterStemmer()
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

# Tokenize the patterns and assign labels

In [ ]:
import numpy as np
import nltk
# nltk.download('punkt')
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

# def tokenize(sentence):
#     """
#     split sentence into array of words/tokens
#     a token can be a word or punctuation character, or number
#     """
#     return nltk.word_tokenize(sentence)

def tokenize(sentence):
    if isinstance(sentence, str):
        return nltk.word_tokenize(sentence)
    else:
        return nltk.word_tokenize(str(sentence))

def stem(word):
    """
    stemming = find the root form of the word
    examples:
    words = ["organize", "organizes", "organizing"]
    words = [stem(w) for w in words]
    -> ["organ", "organ", "organ"]
    """
    return stemmer.stem(word.lower())


def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    # stem each word
    sentence_words = [stem(word) for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag


In [ ]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

In [ ]:
ignore_words = ['?','.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))


In [ ]:
print("patterns======> " , len(xy))
print("tag=====>",len(tags))
print( "unique stemmed words=======>",len(all_words))

# Create a Bag Of Words

In [ ]:
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """

    sentence_words = [stem(word) for word in tokenized_sentence]

    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [ ]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)


# Define The LSTM Model

In [ ]:
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size):
#         super(LSTMModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.lstm = nn.LSTM(input_size, hidden_size)
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, input):
#         output, _ = self.lstm(input.view(len(input), 1, -1))
#         output = self.fc(output.view(len(input), -1))
#         return output



In [ ]:
class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GRUModel, self).__init__()
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input):
        output, _ = self.gru(input.unsqueeze(0))
        output = self.fc(output.squeeze(0))
        return output

# Define Hyperparameters

In [ ]:
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)


# CustomDataset Data 

In [ ]:
class CustomDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [ ]:
dataset = CustomDataset()
train_loader = DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True,num_workers=0)

# Set Device To GPU If Available

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define Loss Function And Optimizer

In [ ]:
model = GRUModel(input_size, hidden_size, output_size).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop

In [ ]:
accuracy_values = []
loss_values = []

In [ ]:
for epoch in range(num_epochs):
    total = 0
    correct = 0
    running_loss = 0.0
    
    for (words, labels) in train_loader:
        words = words.to(torch.float32).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(words)
        _, predicted = torch.max(outputs.data, 1)

        # Convert labels to Long data type
        labels = labels.long()

        # Compute loss
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * words.size(0)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Compute epoch accuracy and loss
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_accuracy = 100 * correct / total
    
    # Append accuracy and loss values to lists
    accuracy_values.append(epoch_accuracy)
    loss_values.append(epoch_loss)
    
    # Print epoch accuracy and loss
    print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {epoch_accuracy:.2f}%, Loss: {epoch_loss:.4f}")
    


print(f'Final Accuracy: {epoch_accuracy:.2f}')
print(f'Final Loss: {epoch_loss:.4f}')

# Save The Model

In [ ]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "Byan_Model_gru.pth"
torch.save(data, FILE)

print(f'Training complete. File saved to {FILE}')

# Plot Accuracy

In [ ]:
plt.plot(range(1, num_epochs+1), accuracy_values)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy')
plt.show()

# Plot loss

In [ ]:

plt.plot(range(1, num_epochs+1), loss_values)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

# Test ChatBot

In [2]:
FILE = "Byan_Model_gru.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']

tags = data['tags']
model_state = data["model_state"]


model = GRUModel(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

conversation = ""
user_name = "You: "
bot_name = "Kharbush: "

while True:
    try:
        user_input = input(f"{user_name}")
    except:
        continue
        
    sentence = user_input
    sentence = tokenize(sentence)
    
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]

            
    if prob.item() > 0.75:
        for intent in intents['intents']:
        #print(intent)
                    
            if tag == intent["tag"]:
                my_resbonce = random.choice(intent['responses'])
                #resbonce = MyTranslate_ToAr(resbonce)
                print(f"\n{bot_name}: {my_resbonce}")
                
                
                break
    else:
        print(f"{bot_name}: I do not understand...") 
    print("\n========== ",prob.item()," =========\n")

NameError: name 'torch' is not defined

In [3]:
import difflib

def autocorrect(input_sentence, stored_sentences):
    if input_sentence is None:
        return "Sorry, I couldn't understand your input."
    
    similarity_scores = []
    for stored_sentence in stored_sentences:
        similarity_scores.append(difflib.SequenceMatcher(None, input_sentence, stored_sentence).ratio())
    max_score = max(similarity_scores)
    if max_score >= 0.5:
        index = similarity_scores.index(max_score)
        return stored_sentences[index]
    else:
        return "Sorry, I couldn't find a match for your sentence."

In [5]:
import json
file_path ='new_data_Ai.json'
with open(file_path, 'r',encoding="utf-8") as f:
    intents = json.load(f)

patterns_l = [pattern.lower() for intent in intents['intents'] for pattern in intent['patterns']]
patterns_c = [pattern for intent in intents['intents'] for pattern in intent['patterns']]


In [ ]:
while True:
    # sentence = "do you use credit cards?"
    user_input = input("You: ")

    patterns_l2 = autocorrect(user_input,patterns_l)
    patterns_c2 = autocorrect(user_input,patterns_c)
    
    if (patterns_l2 in patterns_l) or (patterns_c2 in patterns_c):
        print("ggggggggg = ",patterns_l2)
        print("ggggggggg = ",patterns_c2)

You: hi
ggggggggg =  hi
ggggggggg =  hi
You: long does casting take?
ggggggggg =  mention some clustering methods
ggggggggg =  mention some clustering methods
You: Do you take credit
ggggggggg =  do you like me?
ggggggggg =  Do you like me?
You: Can I pay with
ggggggggg =  Sorry, I couldn't find a match for your sentence.
ggggggggg =  Please can I talk with you?


In [ ]:
if (patterns_l2 in patterns_l) or (patterns_c2 in patterns_c):
        print("ggggggggg = ",sentence)